In [27]:
# Import necessary modules
from typing import Annotated
from typing_extensions import TypedDict

In [28]:
# Import tools from langchain_community
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun

In [29]:
# Initialize Arxiv API wrapper with specific parameters
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)

In [30]:
# Create a tool for querying Arxiv
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [31]:
# Initialize Wikipedia API wrapper with specific parameters
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)

In [32]:
# Create a tool for querying Wikipedia
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [33]:
# Example usage of the Wikipedia tool
wiki_tool.invoke("who is Messi?")

'Page: Lionel Messi\nSummary: Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987) is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one'

In [34]:
# Example usage of the Arxiv tool
arxiv_tool.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with "

In [35]:
# List of tools to be used
tools = [wiki_tool]

In [36]:
# Import necessary modules from langgraph
from langgraph.graph.message import add_messages

In [37]:
# Define the state structure using TypedDict
class State(TypedDict):
    # 'messages' is a list that will accumulate messages
    messages: Annotated[list, add_messages]

In [38]:
# Import StateGraph and constants from langgraph
from langgraph.graph import StateGraph, START, END

In [39]:
# Initialize the state graph with the defined State structure
graph_builder = StateGraph(State)

In [40]:
# Import ChatGroq from langchain_groq
from langchain_groq import ChatGroq

In [41]:
# Import modules for environment variable management
from dotenv import load_dotenv
import os

In [42]:
# Load environment variables from a .env file
load_dotenv()

True

In [43]:
# Retrieve the GROQ API key from environment variables
groq_api_key = os.getenv('GROQ_API_KEY')

In [44]:
# Initialize the ChatGroq model with the API key and model name
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")

In [45]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000236AF1BDFD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000236AEFDEA50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [46]:
# Bind the tools to the language model
llm_with_tools = llm.bind_tools(tools=tools)

In [47]:
# Define the chatbot function that processes the state and returns updated messages
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [48]:
# Import ToolNode and tools_condition from langgraph.prebuilt
from langgraph.prebuilt import ToolNode, tools_condition

In [49]:
# Add the chatbot node to the graph
graph_builder.add_node("chatbot", chatbot)
# Create a ToolNode with the list of tools
tool_node = ToolNode(tools=tools)
# Add the ToolNode to the graph
graph_builder.add_node("tools", tool_node)

In [50]:
# Define conditional edges based on the tools_condition
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

In [51]:
# Define the flow of the graph
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [52]:
# Compile the state graph
graph = graph_builder.compile()

In [53]:
# Import display function for images
from IPython.display import Image, display

In [54]:
# Attempt to display the graph as an image
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [55]:
# Example interaction with the chatbot
user_input = "Hi there!, My name is John"

In [56]:
# Stream events through the graph based on user input
events = graph.stream(
    {"messages": [("user", user_input)]}, stream_mode="values"
)

In [57]:
# Process and display each event's messages
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there!, My name is John
================================== Ai Message ==================================
Tool Calls:
  wikipedia (call_kktq)
 Call ID: call_kktq
  Args:
    query: John


c:\Users\emon1\AppData\Local\Programs\Python\Python311\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\emon1\AppData\Local\Programs\Python\Python311\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


================================= Tool Message =================================
Name: wikipedia

No good Wikipedia Search Result was found
================================== Ai Message ==================================

Well, John is a very common name!


In [58]:
# Another example interaction
user_input = "what is RLHF."

In [59]:
# Stream events through the graph based on new user input
events = graph.stream(
    {"messages": [("user", user_input)]}, stream_mode="values"
)

In [60]:
# Process and display each event's messages
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

what is RLHF.
================================== Ai Message ==================================
Tool Calls:
  wikipedia (call_dd7a)
 Call ID: call_dd7a
  Args:
    query: RLHF
================================= Tool Message =================================
Name: wikipedia

Page: Reinforcement learning from human feedback
Summary: In machine learning, reinforcement learning from human feedback (RLHF) is a technique to align an intelligent agent with human preferences. It involves training a reward model to represent preferences, which can then be used to train other mo
================================== Ai Message ==================================

Reinforcement learning from human feedback (RLHF) is a technique to align an intelligent agent with human preferences. It involves training a reward model to represent preferences, which can then be used to train other models to behave in ways that maximize thos